In [ ]:
#References for this model 
#https://www.tensorflow.org/recommenders/examples/quickstart 
#https://www.kaggle.com/code/mfaaris/hybrid-and-tensorflow-recommender-system

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from tensorflow import feature_column
from typing import Dict, Text
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('normalized_sample.csv')
df.columns

Index(['Brand Name', 'Product Name', 'Price', 'Size', 'Highlights',
       'Long Description', 'Rating', 'Product ID', 'Loves Count', 'Reviews',
       'Ingredients', 'Link', 'Skincare Concern', 'Highlighted Ingredients',
       'Ingredient Callouts', 'Skin Type', 'Skincare Concern Keyword',
       'Highlighted Ingredients Keyword', 'Ingredient Callouts Keywords',
       'Skin Type Keywords', 'Highlights Keywords', 'Bayesian Average'],
      dtype='object')

In [3]:
ratings_df = df[['Brand Name', 'Product Name',
        'Product ID', 'Skincare Concern Keyword',
       'Highlighted Ingredients Keyword', 'Ingredient Callouts Keywords',
       'Skin Type Keywords', 'Highlights Keywords', 'Bayesian Average']]
ratings_df.fillna("unknown", inplace=True)

ratings_df.head(20)

C:\Users\jasks\anaconda3\lib\site-packages\pandas\core\frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,Brand Name,Product Name,Product ID,Skincare Concern Keyword,Highlighted Ingredients Keyword,Ingredient Callouts Keywords,Skin Type Keywords,Highlights Keywords,Bayesian Average
0,Lancôme,Absolue Creme Riche,P477189,"fine,lines,wrinkles,dryness,dullness,formulati...",unknown,unknown,"normal,dry,combination,oily,skincare","refill,available,plumping,good,loss,firmness,g...",4.238116
1,tarte,Shape Tape™ Moisturizer,P476735,unknown,"phytoendorphin,complex,fights,signs,aging,rich...","free,parabens,phthalates,mineral,oil,sulfates,...",unknown,"hydrating,good,dryness,good,antiaging,vegan,cr...",4.276578
2,Clarins,NutriLumière Day,P478246,"fines,lines,wrinkles,dullness,loss,firmness,el...","horse,chestnut,supports,skin,rsquo,micronutrie...","free,parabens,phthalates,mineral,oil,oxybenzon...","normal,dry,combination,oily,skincare","without,sulfates,sls,sles,without,silicones,wi...",4.199655
3,Clarins,Super Restorative Décolleté and Neck Concentrate,P478248,"dark,spots,fines,lines,wrinkles,loss,firmness,...","harungana,visibly,reduces,wrinkles,montpellier...","free,parabens,phthalates,mineral,oil,oxybenzon...","normal,dry,combination,oily,skincare","without,sulfates,sls,sles,without,silicones,wi...",4.276578
4,Erno Laszlo,Firming Cream with Bakuchiol & Spirulina,P478264,"fine,lines,wrinkles,dryness,loss,firmness,elas...","bakuchiol,alternative,retinol,visibly,improves...","free,parabens,phthalates,mineral,oil,sulfates,...","normal,dry,combination,oily,skincare","without,sulfates,sls,sles,without,phthalates,w...",4.199655
5,Clarins,NutriLumière Night,P478245,"fines,lines,wrinkles,dullness,loss,firmness,el...","horse,chestnut,supports,skin,rsquo,micronutrie...","free,parabens,phthalates,mineral,oil,oxybenzon...","normal,dry,combination,oily,skincare","without,sulfates,sls,sles,without,silicones,wi...",4.303371
6,Lancôme,Absolue Revitalizing & Brightening Soft Cream ...,P468375,"fine,lines,wrinkles,dryness,dullness,formulati...","grand,rose,extracts,lanc,rose,extract,rose,ess...","free,parabens,mineral,oil,sulfates,sls,sles","normal,dry,combination,oily,skincare","cream,formula,good,uneven,texture,hydrating,go...",4.303371
7,Hourglass,Equilibrium™ Restorative Hydrating Cream,P476686,"fine,lines,wrinkles,uneven,texture","cell,balancing,complex,trade,helps,nourish,sup...","free,parabens,phthalates,mineral,oil,sulfates,...","normal,dry,combination,oily,skincare","good,antiaging,good,dryness,good,uneven,textur...",4.328251
8,Biotherm,Aquasource Hyalu Plump Moisturizer Gel,P477140,"dryness,redness,dullness,formulation,lightweig...","hyaluronic,acid,delivers,multidepth,moisturiza...","free,parabens,phthalates,mineral,oil,sulfates,...","normal,combination,oily,skincare","plumping,hyaluronic,acid,hydrating,without,sul...",4.256819
9,Yves Saint Laurent,Pure Shots Perfect Plumper Face Cream Refill,P454080,"dryness,fine,lines,wrinkles,loss,firmness,elas...","pure,antiaging,ribose,helps,improve,appearance...","free,sulfates,sls,sles,parabens,formaldehydes,...","dry,skincare",unknown,4.328251


In [4]:
products_df = df[['Product ID', 'Product Name']]
products_df.head()

,Product ID,Product Name
0,P477189,Absolue Creme Riche
1,P476735,Shape Tape™ Moisturizer
2,P478246,NutriLumière Day
3,P478248,Super Restorative Décolleté and Neck Concentrate
4,P478264,Firming Cream with Bakuchiol & Spirulina


In [5]:
ratings_df['Skin Type Keywords'] = ratings_df['Skin Type Keywords'].astype(str)

#this part is turning the data into a tensorflow dataframe
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['Skin Type Keywords', 'Product Name', 'Bayesian Average',"Skincare Concern Keyword"]]))
products = tf.data.Dataset.from_tensor_slices(dict(products_df[['Product Name']]))

ratings = ratings.map(lambda x: {
    "Product Name": x["Product Name"],
    "Skin Type Keywords": x["Skin Type Keywords"],
    "Bayesian Average": float(x["Bayesian Average"]),
    "Skincare Concern Keyword": x["Skincare Concern Keyword"]
                             
})

products = products.map(lambda x: x["Product Name"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<ipython-input-5-19d8c76289fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings_df['Skin Type Keywords'] = ratings_df['Skin Type Keywords'].astype(str)


In [6]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(35_000)
test = ratings.skip(35_000).take(8_188)

Total Data: 81


In [7]:
#creates batches of product names and users 
product_name = products.batch(1_000)
user = ratings.batch(1_000).map(lambda x: x["Skin Type Keywords"])

unique_product_name = np.unique(np.concatenate(list(product_name)))
unique_user = np.unique(np.concatenate(list(user)))

print('Unique Products: {}'.format(len(unique_product_name)))
print('Unique Users: {}'.format(len(unique_user)))

Unique Products: 81
Unique Users: 11


In [8]:
class ProductModel(tfrs.models.Model):
    def __init__(self, rating_weight: float, retrieval_weight: float) -> None:


        super().__init__()

        embedding_dimension = 64

        # User and product models
        self.product_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_product_name, mask_token=None),
          tf.keras.layers.Embedding(len(unique_product_name) + 1, embedding_dimension)
        ])
        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_user, mask_token=None),
          tf.keras.layers.Embedding(len(unique_user) + 1, embedding_dimension)
        ])

        #These are the layers of the model, the last layer has 1 neuron and predicts the rating of a product
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1),
        ])
        
        
        #This task is used to predict the rating of a given user product pair
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        #This task is used to recommend the products
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=products.batch(128).map(self.product_model)
            )
        )

        # These are the weights of the links for each task
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight
        
    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # This creates the user embedding
        user_embeddings = self.user_model(features["Skin Type Keywords"])
        # This creates the product embedding
        product_embeddings = self.product_model(features["Product Name"])

        return (
            user_embeddings,
            product_embeddings,
            # The embeddings are concatenated so they can be inputted to the rating model defined above
            self.rating_model(
                tf.concat([user_embeddings, product_embeddings], axis=1)
            ),
        )
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

        ratings = features.pop("Bayesian Average")

        user_embeddings, product_embeddings, rating_predictions = self(features)

        # Loss for each task
        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )
        retrieval_loss = self.retrieval_task(user_embeddings, product_embeddings)

        # This step combines the losses for each task 
        return (self.rating_weight * rating_loss
                + self.retrieval_weight * retrieval_loss)

In [9]:
model = ProductModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
1/1 [==============================] - 11s 11s/step - root_mean_squared_error: 4.2857 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0123 - factorized_top_k/top_10_categorical_accuracy: 0.0494 - factorized_top_k/top_50_categorical_accuracy: 0.7654 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 374.2924 - regularization_loss: 0.0000e+00 - total_loss: 374.2924
Epoch 2/3
1/1 [==============================] - 0s 154ms/step - root_mean_squared_error: 3.0700 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.2469 - factorized_top_k/top_10_categorical_accuracy: 0.2469 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 363.7651 - regularization_loss: 0.0000e+00 - total_loss: 363.7651
Epoch 3/3
1/1 [==============================] - 0s 141ms/step - root_mean_squared_error: 3.6045 - fact

In [10]:
def predict_product(user, top_n=3):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
      tf.data.Dataset.zip((products.batch(100), products.batch(100).map(model.product_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))


In [11]:
predict_product('normal,dry,combination,oily,skincare', 5)

Top 5 recommendations for user normal,dry,combination,oily,skincare:

1. Mini Facial Treatment Essence (Pitera Essence)
2. SD Advanced ™ PLUS Intensive Moisturizer For Wrinkles & Stretch Marks
3. Rose Deep Hydration Sleeping Mask
4. Prisma Protect
5. Dreamskin Skin Perfector
